# Magyar NLP - előfeldolgozás II.

Az NLP bevezető II. notebookban kiszűrtük a szövegből a zajokat, valamint létrehoztuk a header és a szövegtörsz összefűzéséből a valódi szövegtörzset.
<br>
Vannak olyan lépések, amelyeket el kell végeznünk mielőtt nekiállnánk elemezni a dokumentumgyűjteményünket.
<br>
Ebben a notebook-ban sorrol sorra átvesszük, hogy milyen szövegelemzési előfeldolgozáson kell alávetni a dokomentumgyűjteményt, hogy elemezhető legyen például szózsák modellekkel.
<br>
**Tartalom:**
* Csomagok importálása,
* Fájlok beolvasása,
* Tokenizáció,
* Lemmatizáció,
* Part of Speach (PoS) tagging
* Szűrés lemmákon
* Szűrés a tisztított szövegen
* Szűrt korpusz exportálása

### ---------------------------------------------------------------------------------------


## Csomagok importálása

In [ ]:
# adatkezelés
import pandas as pd
import numpy as np

# NLP csomagok:
import spacy
import hu_core_ud_lg as hu
nlp = hu.load()

from nltk.tokenize import TweetTokenizer 
tknzr = TweetTokenizer() # gyorsabb, mint a Spacy tokenizáció


import warnings
warnings.filterwarnings('ignore')

### ---------------------------------------------------------------------------------------


## Pickle fájl beolvasása

In [ ]:
df_index = pd.read_pickle("2020_corpus_preprocessed.pkl")

In [ ]:
df_index.head(1)

**Fölösleges oszlopok eldobása**

In [ ]:
df_index = df_index.drop(['head', 'szoveg'], axis=1)

### ---------------------------------------------------------------------------------------


## Tokenizáció
Ahhoz, hogy az elemzési egységeket meg tudjuk határozni, a dokumentumok szövegét
tokenizálni kell. Tokennek tekinthető például minden egyes szó egy dokumentumban. “Ez az
alma zöld” mondat 4 tokenből, azaz szóból áll. A tokenizálás sokszor bonyolult feladat
lehet, ha a mondatokban a szavak nem csak szóközökkel vannak elválasztva. A tokenizálás
következtében előáll egy lista a dokumentum szavaiból. Az egyedi szavak leválogatásával
állítható elő a szótár.

In [ ]:
df_index["merged_cleaned_tokens"] = df_index["merged_cleaned"].apply(lambda x: tknzr.tokenize(x))

In [ ]:
df_index[['merged_cleaned', 'merged_cleaned_tokens']][0:2]

### ---------------------------------------------------------------------------------------

## Lemmatizáció
A toldaléklevágás másik módszere a lemmatizálás. A lemmatizálás a nyelvészetben használt
fogalom, és a szó lemmájának, azaz normalizált vagy szótári alakjának megkeresését
szolgálja.

In [ ]:
df_index["merged_cleaned_lemmas"] = df_index['merged_cleaned'].apply(lambda x: [i.lemma_ for i in nlp(x)])

In [ ]:
df_index[['merged_cleaned_tokens', 'merged_cleaned_lemmas']][17:20]

### ---------------------------------------------------------------------------------------

## Part of Speach (PoS) tagging

In [ ]:
df_index['merged_cleaned_ners'] = df_index['merged_cleaned'].apply(lambda x: [i.pos_ for i in nlp(x)])

In [ ]:
df_index[['merged_cleaned_tokens', 'merged_cleaned_lemmas', 'merged_cleaned_ners']][20:25]

### ---------------------------------------------------------------------------------------

## 2020-as NLP feldolgozott korpusz exportálása

In [ ]:
df_index.to_pickle('2020_nlp_preprocessed')

### ---------------------------------------------------------------------------------------

## Dokumentumgyűjtemény szűrése lemmákon

**Feladat**: A 2020-as index.hu-s cikkek közül gyűjtsük le azokat, amelyek a koronavírussal kapcsolatosak.

**1. lépés**: Hozzuk létre a keresendő szavak listáját

In [ ]:
korona = ['vírus', 'pandémia', 'covid', 'koronavírus', 'oltás', 'vakcina', 'karantén']

**2. lépés**: Csináljunk koronavírus flag-et, ezzel jelöljük, hogy van-e a szövegben kapcsolódó szó

In [ ]:
df_index['korona_flg'] = df_index["merged_cleaned_lemmas"].apply(lambda x: 1 if any(i in x for i in korona) else 0)

**3. lépés**: A flag alapján hozzunk létre egy szűrt adatbázist

In [ ]:
df_korona = df_index[df_index['korona_flg'] == 1]

In [ ]:
print("Koronavírushoz köthető cikkek száma a dokumentumgyűjteményben: {} db".format(len(df_korona)))

### ---------------------------------------------------------------------------------------

### Szűrés a tisztított szövegen

Ha például el akarjuk távolítani azokat, amelyek szövegében szerepel az operatív törzs:

In [ ]:
df_korona_no_operativ = df_korona[~df_korona['merged_cleaned'].str.contains('operatív törzs')]

In [ ]:
len(df_korona_no_operativ)

Vagy ha egy lista alapján akarunk szűrni:

In [ ]:
kell = ['operatív', 'magyar', 'vakcina']
df3 = df_korona[df_korona['merged_cleaned'].str.contains(r'\b(?:{})\b'.format('|'.join(kell)))] 
len(df3)

### ---------------------------------------------------------------------------------------

## Szűrt dokumentumgyűjtemény exportálása

In [ ]:
df_korona.to_pickle('2020_korona_corpus')